# Databricks

## Aula 04

Nesta aula iremos trabalhar com o **databricks**, uma plataforma com soluções bastante interessantes para análise de dados!

## Criando um database

Para criar uma base de dados, utilize o comando SQL:

```sql
USE CATALOG alunos;
CREATE DATABASE IF NOT EXISTS seu_nome_de_usuario;
```

**Atenção**: <span style="color:red"> Substitua `seu_nome_de_usuario` por seu nome de usuário Insper!</span>

In [4]:
%sql
-- Seus comandos AQUI!

SyntaxError: invalid syntax (1204441091.py, line 2)

## Ingestão das bases

Para que os dados estejam disponíveis para análise no databricks, será necessário realizar a ingestão dos dados.

Iremos fazer este processo de forma manual. Siga os passos informados pelo professor.

## Exercício 1

Siga os passos demonstrados pelo professor e realize a ingestão das bases de **municipio**.

Caso esteja animado(a), mas totalmente opcional, pode fazer também da **bairros_sp**, **cnae_20_subclasse** e **ocupacao_cbo2002**!

## Importar bibliotecas necessárias

In [ ]:
import pyspark.sql.functions as func

## Carregando tabelas

Para carregar tabelas, utilize:

**Atenção**: <span style="color:red"> Substitua `seu_nome_de_usuario` por seu nome de usuário Insper para considerar o banco de dados correto!</span>

In [ ]:
# Altere seu_nome_de_usuario
spark_df = spark.table("seu_nome_de_usuario.municipio")

Para exibir, utilizamos:

In [ ]:
display(spark_df)

In [ ]:
spark_df.display()

## Exercício 2

Considere o catálogo `bigdata01` e o banco de dados `rais`.

Exiba as tabelas **ocupacao_cbo2002**,  **rais**, **bairros_sp** e **municipio**.

In [ ]:
%sql
USE CATALOG bigdata01;

# Selecionar colunas

No PySpark, podemos utilizar o método **select** passando uma lista de colunas para obter apenas parte das variáveis disponíveis no data frame.

Vamos abrir a tabela `rais` da base `rais` e conferir como filtrar colunas:

Primeiro, ativamos o catálogo de dados:

In [ ]:
%sql
USE CATALOG bigdata01;

Então, podemos ler a tabela `rais.rais` (no formato `database.table`):

In [ ]:
rais_df = spark.table("rais.rais")

E **selecionar colunas** com:

In [ ]:
rais_df.select(["Bairros_SP", "Vl_Remun_Média_SM"])

Perceba no exemplo que solicitamos a exibição de duas colunas.

Mas por que o resultado não foi exibido?

Faltou o uso do `display`!

In [ ]:
rais_df.select(["Bairros_SP", "Vl_Remun_Média_SM"]).display()

# Contando a quantidade de registros

Para sabermos quantos registros (linhas) um Data Frame possui, podemos utilizar:

In [ ]:
rais_df.count()

Vamos exibir em uma string formatada, contando linhas e colunas:

In [ ]:
print(f"{rais_df.count()} Linhas e {len(rais_df.columns)} Colunas")

# Criando novas colunas

Para criar uma nova coluna, podemos chamar o método **withColumn**.

Precisaremos passar o nome da nova coluna e a operação a ser calculada.

Vamos rever o conteúdo da tabela `rais`:

In [ ]:
rais_df.display()

E então, criar uma coluna nova chamada `Vl_Remun_Média_Diária` que irá aproximar o valor pago, em Reais, por dia de trabalho para cada registro da base.

Este é apenas um exemplo! Para o cálculo, consideramos o valor do salário mínimo e a coluna com o valor da remuneração média em salários mínimos.

In [ ]:
salario_minimo_2019 = 998 # em 2019 o salário mínimo foi de R$ 998,00

rais_df = rais_df.withColumn("Vl_Remun_Média_Diária", rais_df["Vl_Remun_Média_SM"] * salario_minimo_2019 / 30.0)

display(rais_df)

## Exercício 3

Crie uma coluna chamada **Rem Primeiro TRI CC** contendo a remuneração total de Janeiro, Fevereiro e Março (ex de coluna: **Vl Rem Janeiro CC**)

# Selecionar linhas

Muitas vezes é necessário realizar um filtro nas linhas, selecionando apenas registros conforme algum critério. Ex: poderíamos querer filtrar apenas os registros da base rais do estado de SP que fossem da cidade de Campinas.

Para realizar um **filtro nas linhas**, utilizaremos o `where`:

In [ ]:
rais_df.where(func.col("CBO_Ocupação_2002") == 411005).show()

Perceba que a saída foi retornada em formato de texto, devido ao uso do `.show()` ao invés de `.display()`!

In [ ]:
rais_df.where(func.col("CBO_Ocupação_2002") == 411005).display()

# Funções de agregação

Muitas vezes é necessário resumir os valores de uma coluna. Para isto, podemos utilizar as funções de cálculo de média, desvio padrão, mínimo, máximo, etc.

No Databricks, podemos fazer isto com:

In [ ]:
rais_df.select("Vl_Remun_Média_Diária").agg({"Vl_Remun_Média_Diária": "skewness"}).collect()[0][0]

Algumas funções para você experimentar:

- `avg`: Média
- `sum`: Soma
- `min`: Valor mínimo
- `max`: Valor máximo
- `count`: Contagem de valores
- `stddev`: Desvio padrão
- `variance`: Variância
- `first`: Primeiro valor
- `last`: Último valor
- `kurtosis`: Curtose
- `skewness`: Assimetria

Podemos também fazer uso do `describe`:

In [ ]:
rais_df.select("Vl_Remun_Média_Diária").describe().display()

In [ ]:
rais_df.describe("Vl_Remun_Média_Diária").display()

E calcular os percentis aproximados com:

In [ ]:
perc = [0.25, 0.5, 0.75]

rais_df.approxQuantile("Vl_Remun_Média_Diária", perc, 0.01)

## Exercício 4

Filtre a base de municípios e descubra o código da cidade de Campinas. Então, utilize o código do município para filtrar a base rais, selecionando apenas os registros da cidade de Campinas.

Com os dados filtrados, calcule a média da coluna `Vl_Rem_Janeiro_CC`.

# Ordenação

Para ordenar a base de dados, utilize **orderBy**

In [ ]:
rais_df.orderBy("Vl_Remun_Média_Diária").display()

# Agrupamentos

Para realizar calculos agrupados por alguma variável, utilize **groupBy**.

Por exemplo, para contar o número de linhas na base de dados, por profissão, utilizamos:

In [ ]:
rais_df.groupBy("CBO_Ocupação_2002").count().display()

Observe que estamos apenas exibindo os resultados. Caso queira, é possível armazenar os resultados em outro DataFrame:

In [ ]:
conta_profissao = rais_df.groupBy("CBO_Ocupação_2002").count()
conta_profissao.display()

Vamos contar quantos municípios diferentes possuem registros de cada profissão. Observe o uso do `countDistinct`:

In [ ]:
rais_df.groupBy("CBO_Ocupação_2002").agg(func.countDistinct("Município")).display()

## Exercício 5

Utilizando a base rais:
- Conte os bairros mais frequentes na base
- Ordene em ordem crescende e decrescente